In [3]:
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# tibanna = Tibanna(env=env)
my_env = 'data'
my_auth = get_key('koray_data')


In [ ]:
# url for repliseq exps
exp_types = ['Repli-seq']
set_url = '/search/?'+'&'.join(['experiments_in_set.experiment_type='+i for i in exp_types])+ \
          '&type=ExperimentSetReplicate&limit=all'
print(set_url)
run_sets = ff_utils.search_metadata(set_url ,key=my_auth)

add_pc = False
add_rel = False
add_wfr = True

counter = 0
all_sets = len(run_sets)
print(str(all_sets)+' total number of sets')

run_sets = [i for i in run_sets if "RepliSeq_Pipeline_0.2.5"  not in i.get('completed_processes', [])]

print(str(all_sets-len(run_sets))+ ' sets completed')


for a_set in run_sets: 
    counter += 1
    print()
    fastqpairs, organism, enzyme, bwa_ref, chrsize_ref, enz_ref, f_size, lab = find_pairs(a_set, my_env, lookfor = 'single')
    if not bwa_ref or not chrsize_ref:
        print(counter, a_set['accession'], organism, enzyme, 'skipping set with not chrsize/bwa index')
        continue  
    print(counter, a_set['accession'], enzyme, organism)      
    # cycle through the experiments
    for exp in fastqpairs.keys():
        if not fastqpairs.get(exp):
            print(exp, 'does not have any fastq files')
            continue
        # Check Part 1 and See if all are okay
        exp_bams = []
        part1 = 'done'
        # part2 = 'done'
        for pair in fastqpairs[exp]:
            #############
            pair_resp = ff_utils.get_metadata(pair, key=my_auth)
            attributions = get_attribution(pair_resp)
            report = get_wfr_out(pair, 'repliseq-parta 0.2.5', my_auth)
            # if run is not successful
            if report['status'].startswith('no'):
                part1 = 'not done'
                if add_wfr:
                    inp_f = {'fastq':pair, 'chromsizes':chrsize_ref, 'bwaIndex':bwa_ref}
                    name_tag = pair.split('/')[2]
                    run_missing_wfr(step_settings('repliseq-parta', organism, attributions), inp_f, name_tag,my_auth, my_env)
            elif report['status'] == 'running':
                part1 = 'still running'
                print('part1 still running')
            # if successful
            else:
                assert report['status'] == 'complete'
                if add_pc:
                    # TODO check if the files already in processed files field.
                    # don't do it if they are already carried there.
                    print('Adding files to preliminary tab')
                    add_preliminary_processed_files(exp,
                                                    [report['bam'], report['bg']],
                                                    my_auth,
                                                    run_type='repliseq')
        # stop progress to part2 
        if part1 is not 'done':
            print(exp, 'has missing Part1 runs')
            part2 = 'not ready'
            part3 = 'not ready'
            continue
        print(exp, 'part1 complete')


In [7]:
# Move files from opc to pc
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.wfr import *

action = False

# get key from keypairs.json
my_key = get_key('koray_data')
print(my_key)
# which sets to move data
set_url = '/search/?experiments_in_set.experiment_type=Repli-seq&type=ExperimentSetReplicate&limit=all'

# exp
# set_url = '/search/?experiment_type=Repli-seq&type=Experiment&limit=all'


# move other processed files to processed files field
def move_opc_to_pc(resp, move_title, con_key):
    opc = resp.get('other_processed_files')
    pc = resp.get('processed_files')
    # if processed_files field already has values, exit
    if pc:
        print('There are files in processed_files field, expected empty')
        return False
    # are there files in opc
    if not opc:
        print('there are no other processed files, skipping')
        return False
    # see if there are other_processed_files to move
    if opc:
        titles = [i['title'] for i in opc]
        if move_title in titles:
            print(resp['accession'], 'files will move')
            move_item = [i for i in opc if i['title'] == move_title]
            assert len(move_item) == 1
            assert move_item[0]['type'] == 'preliminary'
            new_pc = move_item[0]['files']
            new_opc = [i for i in opc if i['title'] != move_title]
            # Time to patch
            patch_data = {}
            add_on = ""
            #if there is something left in opc, patch it, if not delete field
            if new_opc:
                patch_data['other_processed_files'] = opc
            else:
                add_on = 'delete_fields=other_processed_files'
            # patch with processed files
            patch_data['processed_files'] = new_pc
            if action: 
                ff_utils.patch_metadata(patch_data, resp['uuid'], key = con_key, add_on = add_on)
                # update status of pc to status of set or exp
                release_files(resp['uuid'], new_pc, con_key)
            return True
        else:
            return False

        
        

run_sets = ff_utils.search_metadata(set_url , key=my_key)
set_w_apf = 0
exp_w_apf = 0
counter = 0
#move_title = 'HiC Processing Pipeline - Preliminary Files'
move_title = "Repli-Seq Pipeline - Preliminary Files"

print(len(run_sets), 'experiment sets in scope')
for a_set in run_sets:
    set_resp = ff_utils.get_metadata(a_set['uuid'],key=my_key, add_on='frame=raw')
    counter += 1
    print(counter, set_resp['accession'])
    exps = set_resp['experiments_in_set']
    res =  move_opc_to_pc(set_resp, move_title, my_key)
    if res:
        set_w_apf += 1
        print(set_resp['accession'], 'moved to pc')
  
    for exp in exps:
        exp_resp = ff_utils.get_metadata(exp, key=my_key, add_on='frame=raw')
        res_e =  move_opc_to_pc(exp_resp,move_title,my_key)
        if res_e:
            exp_w_apf += 1
            print(exp_resp['accession'], 'moved to pc')
    print()

print(set_w_apf)
print(exp_w_apf)

{'server': 'https://data.4dnucleome.org/', 'key': 'R6ODH4Q2', 'secret': 't4f4r4yq35iaueyx'}
71 experiment sets in scope
1 4DNESO4HIS4U
there are no other processed files, skipping
there are no other processed files, skipping

2 4DNESOE7IW9E
there are no other processed files, skipping
There are files in processed_files field, expected empty

3 4DNESSQCYHJB
there are no other processed files, skipping
There are files in processed_files field, expected empty

4 4DNESRX5DC6I
there are no other processed files, skipping
There are files in processed_files field, expected empty

5 4DNES9IB8HW3
there are no other processed files, skipping
There are files in processed_files field, expected empty
There are files in processed_files field, expected empty

6 4DNESMZPCTLH
there are no other processed files, skipping
There are files in processed_files field, expected empty
There are files in processed_files field, expected empty



KeyboardInterrupt: 

In [ ]:
# Repliseq pipeline part 1 also produces a qc object on the files.
# Change their status to the ones of the files (if the files are in processed files field)

set_url = '/search/?experiments_in_set.experiment_type=Repli-seq&type=ExperimentSetReplicate&limit=all'
run_sets = ff_utils.search_metadata(set_url , key=my_key)

def release_qc(file_id, con_key):
    file_resp = ff_utils.get_metadata(file_id, key=con_key, add_on = 'frame=raw')
    file_status = file_resp['status']
    # skip if file is not released/archived
    if file_status not in ['released', 'released to project', 'archived']:
        return
    qc = file_resp.get('quality_metric')
    if qc:
        qc_resp = ff_utils.get_metadata(qc, key=con_key)
        qc_status = qc_resp['status']
    else:
        return
    
    if qc_status != file_status:
        ff_utils.patch_metadata({'status': file_status}, obj_id=qc, key=con_key)
        print(qc, 'qc object updated with status', file_status)
        return True
    
print(len(run_sets), 'experiment sets in scope')
for a_set in run_sets:
    set_resp = ff_utils.get_metadata(a_set['uuid'],key=my_key, add_on='frame=raw')
    if set_resp.get('processed_files'):
        for a_pf in set_resp['processed_files']:
            release_qc(a_pf, my_key)
            
    exps = set_resp['experiments_in_set']
    for exp in exps:
        exp_resp = ff_utils.get_metadata(exp, key=my_key, add_on='frame=raw')
        if exp_resp.get('processed_files'):
            for a_pf_e in exp_resp['processed_files']:
                release_qc(a_pf_e, my_key)